In [29]:
import pandas as pd
import numpy as np
np.random.seed(0)

In [30]:
data = pd.read_csv("/home/ubuntu/IMDB Dataset (1).csv")

data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [31]:
data['TEXT']=data['review']
data =  data.drop(['review'], axis=1)
data['sentiment'][data['sentiment'] == 'positive'] = 1
data['sentiment'][data['sentiment']== 'negative'] = 0
data['CLASSIFICATION_NUM'] =data['sentiment'].astype('int')
data = data.drop(['sentiment'], axis=1)
data['ID'] = pd.Series(np.arange(1, len(data.index)+1)).astype(str).str.zfill(4)
data.insert(0, 'DATE', pd.datetime.now().replace(microsecond=0))
data.head()



<ipython-input-31-c8410c3e77a6>:8: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  data.insert(0, 'DATE', pd.datetime.now().replace(microsecond=0))


,DATE,TEXT,CLASSIFICATION_NUM,ID
0,2020-09-04 21:54:29,One of the other reviewers has mentioned that ...,1,0001
1,2020-09-04 21:54:29,A wonderful little production. <br /><br />The...,1,0002
2,2020-09-04 21:54:29,I thought this was a wonderful way to spend ti...,1,0003
3,2020-09-04 21:54:29,Basically there's a family where a little boy ...,0,0004
4,2020-09-04 21:54:29,"Petter Mattei's ""Love in the Time of Money"" is...",1,0005


In [32]:
negative_texts = []
positive_texts = []
import re
def clean_text(text):
    text = text.lower()
    
    text = re.sub("@[a-z0-9_]+", ' ', text)
    text = re.sub("[^ ]+\.[^ ]+", ' ', text)
    text = re.sub("[^ ]+@[^ ]+\.[^ ]", ' ', text)
    text = re.sub("[^a-z\' ]", ' ', text)
    text = re.sub(' +', ' ', text)
    return text

In [33]:
for i in range(len(data)):
    if data['CLASSIFICATION_NUM'][i] ==0 :
        negative_texts.append(clean_text(data['TEXT'][i]))
    else:
        positive_texts.append(clean_text(data['TEXT'][i]))

In [34]:
print("عدد النصوص الإيجابية:")
print(len(positive_texts))
print("عدد النصوص السلبية:")
print(len(negative_texts))

عدد النصوص الإيجابية:
25000
عدد النصوص السلبية:
25000


In [35]:
positive_labels = [1]*len(positive_texts) 
negative_labels = [0]*len(negative_texts)  

all_texts = positive_texts + negative_texts 
all_labels = positive_labels + negative_labels

In [36]:
print("عدد النصوص يساوي عدد التصنيفات؟")
print(len(all_labels) == len(all_texts))

عدد النصوص يساوي عدد التصنيفات؟
True


In [37]:
from sklearn.utils import shuffle 
all_texts, all_labels = shuffle(all_texts, all_labels)

In [38]:
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test = train_test_split(all_texts, all_labels, test_size=0.20)

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', min_df=1)
vectorizer.fit(x_train)
x_train = vectorizer.transform(x_train)

In [40]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

In [41]:
from sklearn.metrics import accuracy_score
x_test = vectorizer.transform(x_test)
predictions = model.predict(x_test)
print(" نسبة الصحة باستخدام خوارزمية نايف بيز :")
print(accuracy_score(y_test, predictions))

 نسبة الصحة باستخدام خوارزمية نايف بيز :
0.8374


In [42]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(x_train, y_train)

C:\Users\M_abs\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC()

In [43]:
predictions = model.predict(x_test)
print("نسبة الصحة باستخدام خوارزمية إس في إم :")
print(accuracy_score(y_test, predictions))

نسبة الصحة باستخدام خوارزمية إس في إم :
0.8674


In [44]:
import pickle
with open('model.pickle', 'wb') as file:
    pickle.dump(model, file)
with open('vectorizer.pickle', 'wb') as file:
    pickle.dump(vectorizer, file)

In [45]:
with open('model.pickle', 'rb') as file:
    model = pickle.load(file)
with open('vectorizer.pickle', 'rb') as file:
    vectorizer = pickle.load(file)

In [46]:
example_test = 'I am so happy, the picnic was amazing and the weather was great'
cleaned_example_test = clean_text(example_test)
example_test_vector = vectorizer.transform([cleaned_example_test])
example_result = model.predict(example_test_vector)
print("تصنيف الجملة:", example_test)
print(example_result[0])

تصنيف الجملة: I am so happy, the picnic was amazing and the weather was great
1
